# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

### Agora vamos atualiza a nossa base de preços com as novas cotações

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

# Abrir o navegador
navegador = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 90
Current browser version is 93.0.4577.63 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe


In [ ]:
# Pesquisar e pegar a cotação do Dolar 
navegador.get('https://www.google.com.br/?gws_rd=ssl')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Pegando informação do Google
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

#click() --> Método para clicar em algo na web
#send_keys --> Método para escrecer algo na web

In [ ]:
# Pesquisar e pegar a cotação do Euro
navegador.get('https://www.google.com.br/?gws_rd=ssl')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# Pegando informação do Google
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

In [ ]:
# Entrar no site 'Melhor Cambio'
# Pegar a cotação do Ouro
site = 'https://www.melhorcambio.com/ouro-hoje'
navegador.get(site)
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

In [ ]:
tabela_produtos = pd.read_excel('Produtos.xlsx')
display(tabela_produtos)

In [ ]:
# Passo 1: Atualizar as cotações
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# Passo 2: Atualizar o 'Preço Base Reais' ; 'Cotação' * 'Preço Base Original'
tabela_produtos['Preço Base Reais'] = tabela_produtos['Preço Base Original'] * tabela_produtos['Cotação'] 

# Passo 3: Atualizar o 'Preço Final' ; 'Preço Base Reais' * 'Ajuste'
tabela_produtos['Preço Final'] = tabela_produtos['Preço Base Reais'] * tabela_produtos['Ajuste'] 
tabela_produtos['Cotação'] = tabela_produtos['Cotação'].map('{:.2f}'.format)
tabela_produtos['Preço Base Reais'] = tabela_produtos['Preço Base Reais'].map('{:.2f}'.format)
tabela_produtos['Preço Final'] = tabela_produtos['Preço Final'].map('{:.2f}'.format)
display(tabela_produtos)

In [ ]:
tabela_produtos.to_excel('Produtos Atualizados.xlsx', index=False)
#index=False ; Não mostrar o indice na exportação do arquivo